# Finalize raw data

In [36]:
path = ('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/')

import os
os.chdir(path)

In [37]:
import pandas as pd

df_raw = pd.read_csv('df_raw.csv')

In [38]:
# sexual orientation 6 and 5 are same category across years
df_raw['sexual_orientation_M'].replace({6:5}, inplace=True)
df_raw['sexual_orientation_F'].replace({6:5}, inplace=True)

# fill male NA with female answers
df_raw['sexual_orientation_M'] = df_raw.apply(
    lambda row: row['sexual_orientation_F'] if np.isnan(row['sexual_orientation_M']) else row['sexual_orientation_M'],
    axis=1)

# lifetime partners
# fill male NA with female answers
df_raw['lifetime_male_partners'] = df_raw.apply(
    lambda row: row['lifetime_female_partners'] if np.isnan(row['lifetime_male_partners']) else row['lifetime_male_partners'],
    axis=1)


# fill in doc_diabtetes with pre-diabetes if they are missing
df_raw['doc_diabetes'] = df_raw.apply(
    lambda row: row['doc_prediabetes'] if np.isnan(row['doc_diabetes']) else row['doc_diabetes'],
    axis=1)


# total cholesterol is better than these two, drop female ortientation lifetime partners, and prediabetes
df_raw.drop(['HDL','LDL', 'sexual_orientation_F', 'lifetime_female_partners', 'doc_prediabetes'], axis=1, inplace=True)

df_raw.rename(columns={"sexual_orientation_M": "sexual_orientation", "lifetime_male_partners": "lifetime_partners"}, inplace=True)


In [39]:
# 'annual_HI' - 1-10 $5,000 increments, 11-$75,000 (only in 2004) 12-Over $20,000, 
# 13-Under $20,000, 14-$75,000 to $99,999, 15-$100,000 and Over
# drop 12, add 1 to all values, then make 14 = 1, then minus 15=14 and 16=15

df_raw['annual_HI'] = df_raw['annual_HI'].add(1)
df_raw['annual_HI'].replace({13:np.nan, 14:1, 15:14, 16:15}, inplace=True)


## Replace all 7='refused' and 9='don't know' with NaN

In [40]:
import numpy as np
# columns that are not continuous:
non_cont = ['race_ethnicity', 'edu_level','gender', 'marital_status', 'doc_diabetes','difficult_hearing',
           'difficult_seeing', 'used_CMH', 'health_insurance', 'doc_asthma', 'doc_overweight', 
            'doc_arthritis', 'doc_CHF', 'doc_CHD', 'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis', 
            'doc_liver_condition', 'doc_thyroid_problem', 'doc_COPD', 'doc_cancer', 'doc_kidney', 
            'broken_hip', 'doc_osteoporosis', 'vigorous_activity', 'moderate_activity','doc_sleeping_disorder', 
            'smoker', 'sexual_orientation', 'lifetime_partners', 'doc_HPV', 'alcoholic', 'herpes_2', 'HIV']


for col in non_cont:
    df_raw[col].replace([7,77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw[col].replace([9,99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    
    
# annual_HI - 99 77, used_marijuana - 77 or 777, min_sedetary - 7777
col_2 = ['annual_HI', 'used_marijuana', 'min_sedetary']

for col in col_2:
    df_raw[col].replace([77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw[col].replace([99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan], inplace=True)

## Replace all 1=yes, 2=no with 1=True, 0=False

In [41]:
# all T/F columns
col_TF = ['doc_diabetes', 'difficult_hearing', 'difficult_seeing','used_CMH', 
          'health_insurance', 'doc_asthma', 'doc_overweight', 'doc_arthritis', 'doc_CHF', 'doc_CHD', 
          'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis', 'doc_liver_condition', 'doc_thyroid_problem', 
          'doc_COPD', 'doc_cancer', 'doc_kidney', 'broken_hip', 'doc_osteoporosis', 'vigorous_activity', 
          'moderate_activity','doc_sleeping_disorder', 'smoker', 'doc_HPV', 'alcoholic', 'herpes_2', 'HIV']

# replace 2 with 0
for col in col_TF:
    # should I replace 'indeterminant'/'boarderline' with 0 or NaN?
    df_raw[col].replace([2,3], [0,0], inplace=True)

    
# then need to make all other values in that col either NaN or 0?
# df_raw[col].values[df_raw[col].values > 1] = np.nan

In [42]:
df_raw.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_raw_v2.csv', index=False)